# palaestrAI in a Jupyter Notebook

This is just the equivalent of a system test for the random action experiment.

## Imports

In [1]:
import palaestrai

If the above line works without error, at least the import is done. Thats a first good starter. Now let's import the rest…

In [2]:
import os
import pprint
import tempfile
from pathlib import Path

## Setting the Runtime Configuration

Usually, this is not necessary like this. But because we're running a self-contained test, we want to have a new, fresh database where we want to put it. Which is, in a temporary directory.

In [ ]:
store_dir = tempfile.TemporaryDirectory()
store_dir

We're now going to change palaestrAI's runtime configuration to point to the new directory.

In [ ]:
from palaestrai.core import RuntimeConfig

runtime_config = RuntimeConfig()
runtime_config.reset()
runtime_config.load(
    {"store_uri": "sqlite:///%s/palaestrai.db" % store_dir.name}
)
pprint.pprint(runtime_config.to_dict())

## Create the Database

Next, we create the database at the given URI. It will complain that we're not using TimescaleDB, which is okay.

In [ ]:
from palaestrai.store.database_util import setup_database

setup_database(runtime_config.store_uri)

In [6]:
assert Path("%s/palaestrai.db" % store_dir.name).is_file()

## Run Experiment

In this part, we load our Classic ARL experiment and run it.

In [7]:
experiment_file_path = (
    Path().absolute() / ".." / "fixtures" / "Classic-ARL-Experiment_run-0.yml"
)
assert experiment_file_path.is_file()
os.chdir("../../") # Needed because of palaestrAI shenanigans

In [ ]:
rc = palaestrai.execute(str(experiment_file_path))
assert rc[1].value == 4
rc

If you see something like `('Yo-ho, a dummy experiment run for me!', <ExecutorState.EXITED: 4>)` as output of the previous line, then congratulations, everything went well! Now onwards to the final step…

## Verify Data from the Store

Now that an experiment has been run, there should be something in the store. Note that we don't check whether something *meaningful* is in the store, only that there is *something* in the store.

In [9]:
from sqlalchemy import select

import palaestrai.store
import palaestrai.store.database_model as dbm

In [10]:
dbh = palaestrai.store.Session()

In [ ]:
q = select(dbm.Experiment)
str(q)

In [ ]:
experiment = dbh.execute(q).first()[dbm.Experiment]
experiment.name

In [ ]:
our_experiment_run = experiment.experiment_runs[0]
str(our_experiment_run)

The `document` property of the experiment run object should contain the YAML file, and we should be able to de-searialize it. Let's check that.

In [14]:
assert our_experiment_run.document

In [ ]:
our_experiment_run._document_json

In [ ]:
our_experiment_run.experiment_run_instances[0].uid

… and so on. For a system test, this is enough.